In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('capture20110815-2.pcap.netflow.labeled.csv')

#preprocessing
# df = df[df['Prot']!='IPV6-ICMP']

ips = df['SrcIPAddr:Port'].str.split(':')
df['SrcIPAddr'] = ips.str[0]
df['SrcIPPort'] = ips.str[-1]
ips = df['DstIPAddr:Port'].str.split(':')
df['DstIPAddr'] = ips.str[0]
df['DstIPPort'] = ips.str[-1]

# df.dtypes
# ips = df['Src_IP_Addr:Port'].str.split(':')
# print(pd.sort(ips.str[0:-1]))
# df['Src_IP_Addr'] = ':'.join(ips.str[0:-1])
# df['Src_IP_Port'] = ips.str[-1]
# ips = df['Dst_IP_Addr:Port'].str.split(':')
# df['Dst_IP_Addr'] = ':'.join(ips[0:-1])
# df['Dst_IP_Port'] = ips.str[-1]
df.head()

C:\Users\Anna\AppData\Local\Programs\Python\Python36-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,Bytes,Flows,Label,Labels,SrcIPAddr,SrcIPPort,DstIPAddr,DstIPPort
0,2011-08-15,16:43:20.931,0.235,TCP,147.32.84.59:64131,->,164.8.32.159:54992,PA_,0,3,211,1,Background,NaN,147.32.84.59,64131,164.8.32.159,54992
1,2011-08-15,16:43:20.933,0.444,TCP,147.32.87.252:49218,->,147.32.80.13:3128,FPA_,0,300,19566,1,LEGITIMATE,NaN,147.32.87.252,49218,147.32.80.13,3128
2,2011-08-15,16:43:20.933,4.993,TCP,41.143.58.184:1328,->,147.32.86.179:58675,PA_,0,269,29373,1,Background,NaN,41.143.58.184,1328,147.32.86.179,58675
3,2011-08-15,16:43:20.933,0.187,TCP,147.32.86.155:51273,->,204.154.94.81:80,FA_,0,2,132,1,Background,NaN,147.32.86.155,51273,204.154.94.81,80
4,2011-08-15,16:43:20.933,0.442,TCP,147.32.80.13:3128,->,147.32.87.252:49218,FPA_,0,579,728343,1,LEGITIMATE,NaN,147.32.80.13,3128,147.32.87.252,49218


In [15]:
# df['full_date'] = pd.to_datetime(df['Date_flow_start'] + ' ' + df['time'])

# df.groupby(['Src_IP_Addr']).head()
df.groupby(['SrcIPAddr'])['SrcIPAddr'].agg({'Frequency':'count'}).sort_values(['Frequency'], ascending=False)

C:\Users\Anna\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


,Frequency
SrcIPAddr,
147.32.84.229,76790
147.32.80.9,53721
147.32.84.59,46028
147.32.84.138,20674
147.32.84.118,8113
147.32.85.7,6141
147.32.85.25,3929
147.32.86.20,3806
147.32.84.164,3622
